## Parameters

In [100]:
input_data='./input_data/'
output_data='./output_data/'
cc_output_data='./output_data/CC/'

## Libraries

In [101]:
import numpy as np
import pandas as pd
import datetime

In [102]:
cc_case='WC2'

Load the worst case impact table. 

WC1 and WC2 are the two worst case impact predictions giving reduction in flow by month for 2070.
- *WC1*: Hamududu & Killingtviet 2016
- *WC2*: Harrison & Whittington, 2002

In [103]:
cc_cases=pd.read_csv(input_data + 'climate_change_worst_cases.csv').set_index('Month')
cc_cases

,WC1,WC2
Month,,
10,-0.16,-0.243920
11,-0.20,-0.296011
12,-0.17,-0.396367
1,-0.18,-0.326546
2,-0.16,-0.324661
3,-0.41,-0.395765
4,-0.29,-0.386358
5,-0.24,-0.362866
6,-0.24,-0.350233


In [104]:
daily_flow=pd.read_csv(output_data + 'ngonye_daily.csv')[['Date','Flow']]
daily_flow

,Date,Flow
0,1924-10-01,111.407480
1,1924-10-02,111.407480
2,1924-10-03,111.407480
3,1924-10-04,111.407480
4,1924-10-05,111.407480
...,...,...
33963,2017-09-26,200.552607
33964,2017-09-27,200.552607
33965,2017-09-28,200.552607
33966,2017-09-29,200.552607


In [105]:


daily_flow=daily_flow.set_index(pd.DatetimeIndex(daily_flow['Date']))


daily_flow['Month']=daily_flow.index.month
daily_flow['Year']=daily_flow.index.year
daily_flow=daily_flow.drop('Date',axis=1)
daily_flow.head(4)

,Flow,Month,Year
Date,,,
1924-10-01,111.40748,10,1924
1924-10-02,111.40748,10,1924
1924-10-03,111.40748,10,1924
1924-10-04,111.40748,10,1924


In [106]:
sims=pd.DataFrame(np.arange(1,61,1),columns=['SimNo'])
sims['StartDate']=pd.date_range(start=datetime.date(1933, 10, 1), periods=60, freq=pd.tseries.offsets.DateOffset(years=1))
sims=sims.set_index('SimNo')
sims.head(5)

,StartDate
SimNo,
1,1933-10-01
2,1934-10-01
3,1935-10-01
4,1936-10-01
5,1937-10-01


Calculate the Climate Change Rate which is 0% in 1990 and rises to 100% in 2070. This sets the proportion of the overall climate change impact (by 2070) which is assumed to apply to any given date.

In [107]:
daily_flow['CC_Factor']=daily_flow.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily_flow['Impact_2070']=daily_flow.join(cc_cases,on='Month')[cc_case]
daily_flow['Flow_Adjustment']=1+(daily_flow['CC_Factor']*daily_flow['Impact_2070'])
daily_flow['Flow_Adjusted']=daily_flow['Flow']/daily_flow['Flow_Adjustment']
daily_flow

,Flow,Month,Year,CC_Factor,Impact_2070,Flow_Adjustment,Flow_Adjusted
Date,,,,,,,
1924-10-01,111.407480,10,1924,0.0000,-0.243920,1.000000,111.407480
1924-10-02,111.407480,10,1924,0.0000,-0.243920,1.000000,111.407480
1924-10-03,111.407480,10,1924,0.0000,-0.243920,1.000000,111.407480
1924-10-04,111.407480,10,1924,0.0000,-0.243920,1.000000,111.407480
1924-10-05,111.407480,10,1924,0.0000,-0.243920,1.000000,111.407480
...,...,...,...,...,...,...,...
2017-09-26,200.552607,9,2017,0.3375,-0.293764,0.900855,222.624850
2017-09-27,200.552607,9,2017,0.3375,-0.293764,0.900855,222.624850
2017-09-28,200.552607,9,2017,0.3375,-0.293764,0.900855,222.624850


In [108]:
daily_flow.loc[daily_flow.index=='2013-03-07']

,Flow,Month,Year,CC_Factor,Impact_2070,Flow_Adjustment,Flow_Adjusted
Date,,,,,,,
2013-03-07,3614.802471,3,2013,0.2875,-0.395765,0.886217,4078.911421


In [109]:
tmp=daily_flow.reset_index().head(4018)[['Date','Flow','Flow_Adjusted']].copy()
tmp['Date']=pd.date_range(start=datetime.date(2017, 10, 1), periods=4018, freq='D')
tmp=tmp.set_index('Date')
tmp['Year']=tmp.index.year
tmp['CC_Factor']=tmp.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
tmp

,Flow,Flow_Adjusted,Year,CC_Factor
Date,,,,
2017-10-01,111.407480,111.407480,2017,0.3375
2017-10-02,111.407480,111.407480,2017,0.3375
2017-10-03,111.407480,111.407480,2017,0.3375
2017-10-04,111.407480,111.407480,2017,0.3375
2017-10-05,111.407480,111.407480,2017,0.3375
...,...,...,...,...
2028-09-26,258.797652,258.797652,2028,0.4750
2028-09-27,258.797652,258.797652,2028,0.4750
2028-09-28,256.682789,256.682789,2028,0.4750


In [110]:
daily=daily_flow.append(tmp)
daily

,Flow,Month,Year,CC_Factor,Impact_2070,Flow_Adjustment,Flow_Adjusted
Date,,,,,,,
1924-10-01,111.407480,10.0,1924,0.000,-0.24392,1.0,111.407480
1924-10-02,111.407480,10.0,1924,0.000,-0.24392,1.0,111.407480
1924-10-03,111.407480,10.0,1924,0.000,-0.24392,1.0,111.407480
1924-10-04,111.407480,10.0,1924,0.000,-0.24392,1.0,111.407480
1924-10-05,111.407480,10.0,1924,0.000,-0.24392,1.0,111.407480
...,...,...,...,...,...,...,...
2028-09-26,258.797652,NaN,2028,0.475,NaN,NaN,258.797652
2028-09-27,258.797652,NaN,2028,0.475,NaN,NaN,258.797652
2028-09-28,256.682789,NaN,2028,0.475,NaN,NaN,256.682789


In [111]:
daily.loc[daily.index=='2017-10-01']

,Flow,Month,Year,CC_Factor,Impact_2070,Flow_Adjustment,Flow_Adjusted
Date,,,,,,,
2017-10-01,111.40748,NaN,2017,0.3375,NaN,NaN,111.40748


In [112]:
future=pd.DataFrame(index=pd.date_range(start='10/1/2017', end='09/30/2053', freq='D').rename('Date'))
future['Year']=future.index.year
future['Day']=future.index.day
future['Month']=future.index.month
future['WaterYear']=future.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
future['WaterMonth']=future.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
future=future.astype({'WaterYear': 'int64'})
future=future.astype({'WaterMonth': 'int64'})
future['DateVal']=future.index
future['MonthId']=future['Year']+future['Month']/100
future['WaterDay']=future.apply(lambda x: (x['DateVal']-pd.Timestamp(x['WaterYear'], 10, 1)).days+1,axis=1)
future['WaterWeek']=np.floor((future['WaterDay']-1)/7)+1
future=future.drop('DateVal',axis=1)


future['CC_Factor']=future.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
future

,Year,Day,Month,WaterYear,WaterMonth,MonthId,WaterDay,WaterWeek,CC_Factor
Date,,,,,,,,,
2017-10-01,2017,1,10,2017,1,2017.10,1,1.0,0.3375
2017-10-02,2017,2,10,2017,1,2017.10,2,1.0,0.3375
2017-10-03,2017,3,10,2017,1,2017.10,3,1.0,0.3375
2017-10-04,2017,4,10,2017,1,2017.10,4,1.0,0.3375
2017-10-05,2017,5,10,2017,1,2017.10,5,1.0,0.3375
...,...,...,...,...,...,...,...,...,...
2053-09-26,2053,26,9,2052,12,2053.09,361,52.0,0.7875
2053-09-27,2053,27,9,2052,12,2053.09,362,52.0,0.7875
2053-09-28,2053,28,9,2052,12,2053.09,363,52.0,0.7875


In [113]:

def calc_flows(sim_no):
    if sim_no>=0:
        start_date=sims.loc[sim_no]['StartDate']

        fut=future.copy()

        fut['SampleDate']=pd.date_range(start=start_date, periods=13149, freq='D')
        fut['SampleCC_Factor']=fut.join(daily,on='SampleDate',rsuffix='_daily')['CC_Factor_daily']
        fut['SampleFlowUnadjusted']=fut.join(daily,on='SampleDate',rsuffix='_daily')['Flow']
        fut['SampleFlow']=fut.join(daily,on='SampleDate',rsuffix='_daily')['Flow_Adjusted']
        fut['Impact_2070']=fut.join(cc_cases,on='Month')[cc_case]
        if mml_method:
            fut['Flow']=fut['SampleFlowUnadjusted']*(1+fut['Impact_2070']*(fut['CC_Factor']-fut['SampleCC_Factor']))
        else:
            fut['Flow']=fut['SampleFlow']*(1+fut['Impact_2070']*fut['CC_Factor'])

        fut.to_csv(cc_output_data + 'ngonye_daily_' + cc_case + '_' + str(sim_no) + '.csv')

        sims.at[sim.Index,'Mean_2023_2040']=fut.loc[(fut.index>=datetime.datetime(2023, 10, 1)) & (fut.index<=datetime.datetime(2041, 9, 30))]['Flow'].mean()
        sims.at[sim.Index,'Mean_2023_2052']=fut.loc[(fut.index>=datetime.datetime(2023, 10, 1)) & (fut.index<=datetime.datetime(2053, 9, 30))]['Flow'].mean()


In [114]:
for sim in sims.itertuples():
    mml_method=False
    calc_flows(sim.Index)
    
sims

,StartDate,Mean_2023_2040,Mean_2023_2052
SimNo,,,
1,1933-10-01,935.332432,1006.073676
2,1934-10-01,977.133796,1010.648809
3,1935-10-01,984.615994,1015.168524
4,1936-10-01,1007.841821,1021.473712
5,1937-10-01,1044.625840,1022.378299
6,1938-10-01,1080.270215,1028.629486
7,1939-10-01,1129.133370,1044.361566
8,1940-10-01,1146.991552,1068.189626
9,1941-10-01,1136.610478,1066.557247


In [115]:
sims.to_csv(cc_output_data + 'climate_change_sims_' + cc_case + '.csv')